# Convolution02 

### mnist_cnn
1. 딥러닝 학습시키는 단계
2. 우리가 만들 CNN 구조 확인
3. MNIST에 CNN 적용 코드를 함께 작성

In [1]:
import torch
import torch.nn as nn

In [2]:
inputs = torch.Tensor(1,1,28,28)

In [4]:
inputs.shape

torch.Size([1, 1, 28, 28])

In [5]:
conv1 = nn.Conv2d(1,32,3,padding=1)

In [6]:
pool = nn.MaxPool2d(2)
pool

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)

In [8]:
conv2 = nn.Conv2d(32,64,3,padding=1)

In [9]:
conv2

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

In [10]:
out = conv1(inputs)

In [11]:
out.shape

torch.Size([1, 32, 28, 28])

In [13]:
out = pool(out)

In [14]:
out.shape

torch.Size([1, 32, 14, 14])

In [15]:
out = conv2(out)

In [16]:
out.shape

torch.Size([1, 64, 14, 14])

In [17]:
out = pool(out)

In [18]:
out.shape

torch.Size([1, 64, 7, 7])

In [19]:
print(out.size(0),out.size(1),out.size(2),out.size(3))

1 64 7 7


In [20]:
out = out.view(out.size(0),-1)
out.shape

torch.Size([1, 3136])

In [21]:
fc = nn.Linear(3136,10)
out = fc(out)

In [22]:
out

tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]],
       grad_fn=<AddmmBackward>)

In [23]:
out.shape

torch.Size([1, 10])

위는 임의로 해본 소스 이다. 아래에서는 mnist 를 이용하여 더 디테일 하게 구현해보도록 하겠다

In [66]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms

import torch.nn.init

In [67]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777) # 랜덤벨류 고정
if device == 'cuda' :
    torch.cuda.manual_seed_all(777)

In [68]:
# parameters

learning_rate = 0.001
training_epochs = 15
batch_size = 100

In [69]:
# MNIST dataset 
# 이부분은 동일하게 계속 써야 MNIST를 다시 받지 않고 사용 가능합니다. 
mnist_train = dsets.MNIST(root='MNIST_data/',
                         train = True,
                         transform = transforms.ToTensor(),
                         download=True)
mnist_test = dsets.MNIST(root = 'MNIST_data/',
                        train= False,
                        transform = transforms.ToTensor(),
                         download=True
                        )

In [70]:
# data_loader 만들기
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size = batch_size,
                                          shuffle = True,
                                          drop_last = True)

In [83]:
# CNN Model 생성

class CNN(nn.Module) :
    def __init__(self) :
        super(CNN,self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1,32,kernel_size=3,stride=1,padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.layer2  = nn.Sequential(
            nn.Conv2d(32,64,kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.fc = nn.Linear(7*7*64, 10, bias = True)
        torch.nn.init.xavier_uniform_(self.fc.weight)
        # xavier_uniform ???
        
    def foward(self, x) :
        print(x)
        out = self.layer1(x)
        out = self.layer2(out)
        
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

In [90]:
model= CNN().to(device)

value = torch.Tensor(1,1,28,28).to(device)
print((model(value)).shape)

NotImplementedError: 

In [85]:
device

'cuda'

In [86]:
model

CNN(
  (layer1): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (layer2): Sequential(
    (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Linear(in_features=3136, out_features=10, bias=True)
)

In [87]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr = learning_rate)

In [89]:
# traning
total_batch = len(data_loader)

for epoch in range(training_epochs) :
    avg_cost = 0
    
    for X,Y in data_loader :
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad() # 중요 반드시 넣어라
        hypothesis = model(X)
        
        cost = criterion(hypothesis, Y)
        cost = backward()
        optimizer.step()
        
        avg_cost += cost/ total_batch
        
    print('[Epoch:{}] cost = }', format(epoch+1),avg_cost)
    
print('Learning Finish')

NotImplementedError: 